In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import random

# 1. Шифр Цезаря
def caesar_cipher(text, shift):
    result = ''
    for char in text:
        if 'a' <= char <= 'z':
            start = ord('a')
            shifted_char = chr((ord(char) - start + shift) % 26 + start)   # шифрование
        elif 'A' <= char <= 'Z':
            start = ord('A')
            shifted_char = chr((ord(char) - start + shift) % 26 + start)
        else:
            shifted_char = char  # Оставляем не-буквенные символы без изменений
        result += shifted_char
    return result
# start = ord('a'):  Если символ является строчной буквой, эта строка получает ASCII-код буквы 'a' и сохраняет его   
#   % 26:  dычисляет остаток от деления на 26 (количество букв в английском алфавите)
#  ord(char) получает ASCII-код текущего символа
# ord(char) - start  смещение текущего символа относительно начала алфавита 
#   + start:  Прибавляет ASCII-код 'a' обратно, чтобы получить ASCII-код зашифрованного символа
# chr(...): Преобразует ASCII-код обратно в символ
# shifted_char = ...: Сохраняет зашифрованный символ в переменной shifted_char

In [6]:
original_text = " Hello first task "
shift = 3
encrypted_text = caesar_cipher(original_text,shift) 
print(f"Original: {original_text}, Encrypted: {encrypted_text}")


Original:  Hello first task , Encrypted:  Khoor iluvw wdvn 


In [7]:
# 2. Подготовка данных
alphabet = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ " # Пробел тоже
char_to_index = {char: index for index, char in enumerate(alphabet)}
index_to_char = {index: char for index, char in enumerate(alphabet)} # Создает обратный словарь 
vocab_size = len(alphabet)
max_len = 20 # Максимальная длина фразы

def create_dataset(num_samples, shift):   # набор данных для обучения (kоличество пар,bеличина сдвига)
    pairs = []
    for _ in range(num_samples):
        phrase = ''.join(random.choice(alphabet) for _ in range(random.randint(5, max_len))) # Генерирует случайную фразу:случайная длина
        encrypted_phrase = caesar_cipher(phrase, shift)  # Шифрует сгенерированную фразу 
        pairs.append((encrypted_phrase, phrase))
    return pairs

shift_amount = 3 # Сдвиг Цезаря
training_data = create_dataset(1000, shift_amount) # 1000 примеров

def phrase_to_tensor(phrase):  # преобразует фразу в тензор PyTorch, используя one-hot encoding
    tensor = torch.zeros(len(phrase), vocab_size)  # Создает тензор PyTorch, заполненный нулями 
    for i, char in enumerate(phrase):
        index = char_to_index[char]
        tensor[i, index] = 1 # one-hot encoding
    return tensor
# char_to_index Создает словарь, который отображает каждый символ в алфавите на его индекс   
# enumerate(alphabet) возвращает пары (индекс, символ)


In [8]:
print (training_data)  

[('BcVAIHCy', 'YzSXFEZv'), ('tsTrWKwEGkPu', 'qpQoTHtBDhMr'), ('kuPuOZQOBYrB', 'hrMrLWNLYVoY'), ('PrxvzLlEJqwS', 'MouswIiBGntP'), ('CMykvOXcEKZ', 'ZJvhsLUzBHW'), (' KSpXe', ' HPmUb'), ('kZwxxvmoBsRJDQrMB', 'hWtuusjlYpOGANoJY'), ('cHfKuNfTWqswGljbf', 'zEcHrKcQTnptDigyc'), ('BunjGOQkpyEfUCs', 'YrkgDLNhmvBcRZp'), ('VhJbQivfLPOvNxa', 'SeGyNfscIMLsKux'), ('xEjAXMBn', 'uBgXUJYk'), ('hKGfgtqCPdilo', 'eHDcdqnZMafil'), ('ekgQKfxi Mvoj', 'bhdNHcuf Jslg'), ('DyeFSb', 'AvbCPy'), ('ehRRXZ', 'beOOUW'), ('WhYyiHZYRl', 'TeVvfEWVOi'), ('iu pAkhQ nuztmqqh', 'fr mXheN krwqjnne'), ('sUbwDOq', 'pRytALn'), ('flOsAVENCILB', 'ciLpXSBKZFIY'), ('iftQAsbjdAvicfXO', 'fcqNXpygaXsfzcUL'), ('VnFoxdIxyB', 'SkCluaFuvY'), ('obmvAOvN', 'lyjsXLsK'), ('XrJQvFZ khTmmeoCqsUv', 'UoGNsCW heQjjblZnpRs'), ('oIIdHMIgkOiroz', 'lFFaEJFdhLfolw'), ('LnOXrmMXXXYzESxi', 'IkLUojJUUUVwBPuf'), ('XuCMIhjR', 'UrZJFegO'), ('ScgaQSBxdj ajQjTCyl', 'PzdxNPYuag xgNgQZvi'), ('ZErbGsgd', 'WBoyDpda'), ('EHMmuP', 'BEJjrM'), ('wvKMXCfOtpA', 'tsHJUZcL

In [9]:
# 3. Архитектура RNN
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRNN, self).__init__()  # экземпляр класса SimpleRNN (простая рекуррентная нейронная сеть)
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True) # batch_first=True
        self.linear = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Инициализация скрытого состояния нулями
        h0 = torch.zeros(1, x.size(0), self.hidden_size).to(x.device) # (num_layers * num_directions, batch, hidden_size)
        out, _ = self.rnn(x, h0)
        out = self.linear(out)
        return out


In [10]:
# Обучение
input_size = vocab_size
hidden_size = 150
output_size = vocab_size # Определяет размер выходного слоя модели равным размеру словаря. 
# Для каждого символа во входной фразе модель будет предсказывать вероятность для каждого символа в алфавите
model = SimpleRNN(input_size, hidden_size, output_size)

criterion = nn.CrossEntropyLoss() # Определяет функцию потерь
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train(model, training_data, optimizer, criterion, epochs=10):
    model.train()
    for epoch in range(epochs):
        epoch_loss = 0
        for encrypted_phrase, original_phrase in training_data:
            # Преобразование в тензоры
            input_tensor = phrase_to_tensor(encrypted_phrase).unsqueeze(0) # Добавляем размерность батча
            target_tensor = torch.tensor([char_to_index[char] for char in original_phrase]) # Преобразует оригинальную фразу в тензор, 
            # содержащий индексы символов

            # Прямой проход
            output = model(input_tensor)
 # output будет содержать предсказанные вероятности для каждого символа в каждой позиции       
            # Вычисление потерь
            loss = criterion(output.squeeze(0), target_tensor) # Убираем размерность батча

            # Обратное распространение и оптимизация
            optimizer.zero_grad()  # oбнуляем градиенты параметров модели
            loss.backward()  # вычисляем градиенты функции потерь по отношению к параметрам модели
            optimizer.step() # Обновляем параметры модели

            epoch_loss += loss.item() # добавляем текущую потерю к общей потере за эпоху

        print(f'Epoch {epoch+1}, Loss: {epoch_loss/len(training_data)}') # выводиv среднюю потерю за эпоху

train(model, training_data, optimizer, criterion)

Epoch 1, Loss: 1.167912140114233
Epoch 2, Loss: 0.013415912774391473
Epoch 3, Loss: 0.003729408350540325
Epoch 4, Loss: 0.0015783788770786487
Epoch 5, Loss: 0.0007758239676477388
Epoch 6, Loss: 0.0004099915699334815
Epoch 7, Loss: 0.00022541512083262206
Epoch 8, Loss: 0.00012683498687692917
Epoch 9, Loss: 7.238545885047642e-05
Epoch 10, Loss: 4.1691229367643244e-05


In [11]:
# Проверка качества (пример)
def decrypt(model, encrypted_phrase):
    model.eval()
    with torch.no_grad():
        input_tensor = phrase_to_tensor(encrypted_phrase).unsqueeze(0)
        output = model(input_tensor) # (1, seq_len, vocab_size)
        predicted_indices = torch.argmax(output, dim=2).squeeze(0)  # (seq_len)
        decrypted_phrase = ''.join([index_to_char[index.item()] for index in predicted_indices]) # Corrected
    return decrypted_phrase

# Пример использования
test_phrase = "Khoor iluvw wdvn"
decrypted_phrase = decrypt(model, test_phrase)
print(f"Encrypted: {test_phrase}")
print(f"Decrypted: {decrypted_phrase}")

Encrypted: Khoor iluvw wdvn
Decrypted: Hello first task
